#### Importation des Bibliothèques

In [1]:
import requests
import pandas as pd
import numpy as np

#### Définition des Indicateurs

In [2]:
# Définir les indicateurs 
indicators = {
    "EG.FEC.RNEW.ZS": "Taux d'énergie renouvelable",
    "EG.ELC.RNWX.ZS": "Investissements en énergies renouvelables (% du PIB)",
    "NY.GDP.PCAP.CD": "PIB par habitant (USD)",
    "SE.SEC.NENR": "Taux de scolarisation secondaire (%)",
    "EG.ELC.ACCS.ZS": "Accès à l'électricité (%)"
}


# Définir la période de 2010 à 2023
start_year = 2010
end_year = 2023







#### Ce code récupère les données des indicateurs  via l'API de la **Banque mondiale** pour la période **2010-2023**. Il combine ces données dans un **DataFrame unique** en effectuant les étapes suivantes :

1. Envoi d'une requête pour chaque indicateur.  
2. Normalisation des données JSON en un tableau structuré.  
3. Sélection et renommage des colonnes essentielles (Pays, Année, Valeur).  
4. Fusion progressive des données sur les colonnes "Country" et "Year".  


In [3]:
# DataFrame vide pour combiner les données
df_combined = pd.DataFrame()

# Boucle pour récupérer chaque indicateur et ajouter les données dans le dataframe
for indicator, description in indicators.items():
    
    url = f"http://api.worldbank.org/v2/country/all/indicator/{indicator}?format=json&date={start_year}:{end_year}&per_page=5000"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data and len(data) > 1:
            results = data[1]
            
            # Normaliser la réponse JSON pour en faire un DataFrame
            df = pd.json_normalize(results)
            
            # Garder uniquement les colonnes nécessaires : pays, valeur de l'indicateur et année
            df = df[["country.value", "value", "date"]]
            
            # Renommer les colonnes
            df.columns = ["Country", description, "Year"]
            
            # Si df_combined est vide, initialisez-le avec le premier indicateur
            if df_combined.empty:
                df_combined = df
            else:
                # Fusionner sur les colonnes "Country" et "Year"
                df_combined = pd.merge(df_combined, df, on=["Country", "Year"], how="outer")
        else:
            print(f"Aucune donnée trouvée pour l'indicateur {indicator}.")
    else:
        print(f"Erreur lors de la requête pour l'indicateur {indicator} : {response.status_code}")

#### Vérification de la structure des données récupérées

In [4]:
# Vérification de la structure des données récupérées
print(df_combined.head())

       Country  Taux d'énergie renouvelable  Year  \
0  Afghanistan                         15.2  2010   
1  Afghanistan                         12.6  2011   
2  Afghanistan                         15.4  2012   
3  Afghanistan                         16.9  2013   
4  Afghanistan                         19.1  2014   

   Investissements en énergies renouvelables (% du PIB)  \
0                                                NaN      
1                                                NaN      
2                                                NaN      
3                                                NaN      
4                                                NaN      

   PIB par habitant (USD)  Taux de scolarisation secondaire (%)  \
0              560.621505                                   NaN   
1              606.694676                                   NaN   
2              651.417134                                   NaN   
3              637.087099                               46

#### Vérifier les colonnes extraites

In [5]:
# Vérifier les colonnes extraites
print("\nListe des colonnes disponibles :")
print(df_combined.columns)


Liste des colonnes disponibles :
Index(['Country', 'Taux d'énergie renouvelable', 'Year',
       'Investissements en énergies renouvelables (% du PIB)',
       'PIB par habitant (USD)', 'Taux de scolarisation secondaire (%)',
       'Accès à l'électricité (%)'],
      dtype='object')


#### Identification et Quantification des Valeurs Manquantes par Colonne

In [6]:
# Vérifier les colonnes avec des valeurs manquantes
print("\nValeurs manquantes par colonne :")
print(df_combined.isnull().sum())

# Pourcentage de valeurs manquantes par colonne
print("\nPourcentage de valeurs manquantes par colonne :")
missing_percent = (df_combined.isnull().sum() / len(df_combined)) * 100
print(missing_percent)


Valeurs manquantes par colonne :
Country                                                    0
Taux d'énergie renouvelable                              585
Year                                                       0
Investissements en énergies renouvelables (% du PIB)    2610
PIB par habitant (USD)                                   114
Taux de scolarisation secondaire (%)                    2363
Accès à l'électricité (%)                                305
dtype: int64

Pourcentage de valeurs manquantes par colonne :
Country                                                  0.000000
Taux d'énergie renouvelable                             15.708915
Year                                                     0.000000
Investissements en énergies renouvelables (% du PIB)    70.085929
PIB par habitant (USD)                                   3.061224
Taux de scolarisation secondaire (%)                    63.453276
Accès à l'électricité (%)                                8.190118
dtype: float64


In [7]:
# Renommer les colonnes pour utiliser des noms en français
df_combined.rename(columns={"Country": "Pays", "Year": "Année"}, inplace=True)

# Vérification après renommage
print("Colonnes après renommage :", df_combined.columns)

Colonnes après renommage : Index(['Pays', 'Taux d'énergie renouvelable', 'Année',
       'Investissements en énergies renouvelables (% du PIB)',
       'PIB par habitant (USD)', 'Taux de scolarisation secondaire (%)',
       'Accès à l'électricité (%)'],
      dtype='object')


In [8]:
print(df_combined.dtypes)

Pays                                                     object
Taux d'énergie renouvelable                             float64
Année                                                    object
Investissements en énergies renouvelables (% du PIB)    float64
PIB par habitant (USD)                                  float64
Taux de scolarisation secondaire (%)                    float64
Accès à l'électricité (%)                               float64
dtype: object


#### Gestion des Valeurs Manquantes pour les Colonnes Numériques

In [9]:
# Identifier les colonnes numériques
colonnes_numeriques = df_combined.select_dtypes(include=['float64', 'int64']).columns

# Remplir par la moyenne par pays pour chaque indicateur
for col in colonnes_numeriques:
    df_combined[col] = df_combined.groupby("Pays")[col].transform(lambda x: x.fillna(x.mean()))

# Remplir les valeurs restantes par la moyenne globale
for col in colonnes_numeriques:
    df_combined[col] = df_combined[col].fillna(df_combined[col].mean())


1-**Remplir les valeurs manquantes par la moyenne, par pays :**

Pour chaque colonne numérique (col), groupby("Pays") crée un groupe de données par pays.
**transform(lambda x: x.fillna(x.mean()))** remplit les valeurs manquantes dans chaque groupe par la moyenne de ce groupe. Cela permet de compenser les valeurs manquantes de manière locale, selon les données disponibles pour chaque pays.

2-**Remplir les valeurs restantes par la moyenne globale :**

Ensuite, pour les valeurs manquantes restantes après le remplissage par groupe, on remplit encore une fois par la moyenne globale de la colonne.
**fillna(df_combined[col].mean())** remplace les valeurs manquantes par la moyenne calculée sur toute la colonne, assurant ainsi une imputation uniforme pour les données globales.


#### Vérification des valeurs manquantes après traitement

In [10]:
# Vérification des valeurs manquantes après traitement
print("\nValeurs manquantes par colonne après traitement :")
print(df_combined.isnull().sum())

# Pourcentage de valeurs manquantes après traitement
print("\nPourcentage de valeurs manquantes par colonne :")
missing_percent = (df_combined.isnull().sum() / len(df_combined)) * 100
print(missing_percent)


Valeurs manquantes par colonne après traitement :
Pays                                                    0
Taux d'énergie renouvelable                             0
Année                                                   0
Investissements en énergies renouvelables (% du PIB)    0
PIB par habitant (USD)                                  0
Taux de scolarisation secondaire (%)                    0
Accès à l'électricité (%)                               0
dtype: int64

Pourcentage de valeurs manquantes par colonne :
Pays                                                    0.0
Taux d'énergie renouvelable                             0.0
Année                                                   0.0
Investissements en énergies renouvelables (% du PIB)    0.0
PIB par habitant (USD)                                  0.0
Taux de scolarisation secondaire (%)                    0.0
Accès à l'électricité (%)                               0.0
dtype: float64


#### Vérifier s'il y a des doublons

In [11]:
# Vérifier s'il y a des doublons
duplicates = df_combined[df_combined.duplicated()]
print(f"\nNombre de doublons trouvés : {len(duplicates)}")


Nombre de doublons trouvés : 0


In [12]:
# Vérifiez un aperçu des données
print(df_combined.head())

          Pays  Taux d'énergie renouvelable Année  \
0  Afghanistan                         15.2  2010   
1  Afghanistan                         12.6  2011   
2  Afghanistan                         15.4  2012   
3  Afghanistan                         16.9  2013   
4  Afghanistan                         19.1  2014   

   Investissements en énergies renouvelables (% du PIB)  \
0                                           4.838663      
1                                           4.838663      
2                                           4.838663      
3                                           4.838663      
4                                           4.838663      

   PIB par habitant (USD)  Taux de scolarisation secondaire (%)  \
0              560.621505                             47.154018   
1              606.694676                             47.154018   
2              651.417134                             47.154018   
3              637.087099                             46.1

#### Ajouter des colonnes calculer :

In [13]:
# Ajouter la classification par revenu basée sur le PIB par habitant
def classify_income(row):
    if row['PIB par habitant (USD)'] < 1046:
        return 'Low income'
    elif 1046 <= row['PIB par habitant (USD)'] < 4136:
        return 'Lower middle income'
    elif 4136 <= row['PIB par habitant (USD)'] < 12846:
        return 'Upper middle income'
    else:
        return 'High income'

df_combined['Revenu'] = df_combined.apply(classify_income, axis=1)


# Ajouter une colonne binaire pour déterminer si un pays dépasse 50% d'énergie renouvelable
df_combined['Dépassement 50%'] = df_combined['Taux d\'énergie renouvelable'].apply(lambda x: 'Oui' if x > 50 else 'Non')

# Ajouter la classification de la transition énergétique
def classify_transition(row):
    if row['Taux d\'énergie renouvelable'] > 50:
        return 'Avancée'
    elif 25 <= row['Taux d\'énergie renouvelable'] <= 50:
        return 'Intermédiaire'
    else:
        return 'Débutante'

df_combined['Classification Transition Énergétique'] = df_combined.apply(classify_transition, axis=1)

#### Exporter les données dans un fichier CSV

In [14]:
# Filtrer les données pour ne garder que l'année 2023
df_combined_2023 = df_combined[df_combined['Année'] == '2023']

# Enregistrer les données nettoyées et transformées dans un fichier CSV
df_combined_2023.to_csv('transition_energetique_pays_2023.csv', index=False)

print("Données pour l'année 2023 enregistrées dans 'transition_energetique_pays_2023.csv'")

Données pour l'année 2023 enregistrées dans 'transition_energetique_pays_2023.csv'
